In [1]:
import pathlib
import matplotlib.pyplot as plt

from c3.libraries import fidelities
from c3.parametermap import ParameterMap
from utils import *
from c3.experiment import Experiment
import c3.libraries.constants as constants
import c3.utils.qt_utils as qt_utils

2021-08-17 17:12:31.775893: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-17 17:12:31.775938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# plotting functions
def plotOccupations(
        experiment: Experiment,
        populations: np.array,
        gate_sequence: List[str],
        level_names: List[str] = None,
        filename: str = None,
) -> None:
    """
    Plots time dependent populations. They need to be calculated with `runTimeEvolution` first.

    Parameters
    ----------
    experiment: Experiment
        The experiment containing the model and propagators
    populations: np.array
        Population vector for each time step
    gate_sequence: List[str]
        List of gate names that will be applied to the state
    level_names: List[str]
        Optional list of names for the levels. If none, the default list
        from the experiment will be used.
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.

    Returns
    -------

    """
    # plot populations
    fig, axs = plt.subplots(1, 1)
    dt = experiment.ts[1] - experiment.ts[0]
    ts = np.linspace(0.0, dt * populations.shape[1], populations.shape[1])
    axs.plot(ts / 1e-9, populations.T)

    # plot vertical lines
    gate_steps = [experiment.partial_propagators[g].shape[0] for g in gate_sequence]
    for i in range(1, len(gate_steps)):
        gate_steps[i] += gate_steps[i - 1]
    gate_times = gate_steps * dt
    plt.vlines(gate_times / 1e-9, tf.reduce_min(populations), tf.reduce_max(populations),
               linestyles=':', colors="black")

    # set plot properties
    axs.tick_params(direction="in", left=True, right=True, top=False, bottom=True)
    axs.set_xlabel('Time [ns]')
    axs.set_ylabel('Population')
    plt.legend(level_names if level_names else model.state_labels)
    plt.tight_layout()

    # show and save
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


def plotSignal(time, signal, filename=None, spectrum_cut=1e-4) -> None:
    """
    Plots a time dependent signal and its normalised frequency spectrum.

    Parameters
    ----------
    time
        timestamps
    signal
        signal value
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.
    spectrum_cut:
        If not None, only the part of the normalised spectrum will be plotted
        whose absolute square is larger than this value.

    Returns
    -------

    """
    # plot time domain
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].set_title('Signal')
    axs[0].plot(time, signal)
    axs[0].set_xlabel('time')

    # calculate frequency spectrum
    n_samples = time.shape[-1]
    freq_signal = np.fft.rfft(signal)
    normalised = freq_signal / np.max(freq_signal)
    freq = np.fft.rfftfreq(n_samples, time[-1] / n_samples)

    # cut spectrum if necessary
    if spectrum_cut is not None:
        limits = np.flatnonzero(np.abs(normalised) ** 2 > 1e-4)
        freq = freq[limits[0]:limits[-1]]
        normalised = normalised[limits[0]:limits[-1]]

    # plot frequency domain
    axs[1].set_title('Spectrum')
    axs[1].plot(freq, normalised.real, label="Re")
    axs[1].plot(freq, normalised.imag, label="Im")
    axs[1].plot(freq, np.abs(normalised) ** 2, label="Square")
    axs[1].set_xlabel('frequency')
    axs[1].legend()

    # show and save
    plt.tight_layout()
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()

In [3]:
# preparation
active_levels = 4
occupied_levels = [0]
directory = "./output"
output_dir = pathlib.Path(directory)
output_dir.mkdir(parents=True, exist_ok=True)

# model
q1 = createQubit(1, 5, 5e9, -300e6)
model = createModel([q1])
generator = createGenerator(model)

# gate
t_final = 10e-9
envelope = createGaussianPulse(t_final, t_final / 4)
ideal = qt_utils.np_kron_n([
    constants.Id,
    constants.X,
])
gate = createSingleQubitGate("lower-X", t_final, 5e9, envelope, model, q1, ideal)
gates = [gate]
gate_names = list(map(lambda g: g.get_key(), gates))

# experiment
exp = Experiment(pmap=ParameterMap(instructions=gates, model=model, generator=generator))
exp.set_opt_gates(gate_names)
unitaries = exp.compute_propagators()
print('unitaries: ', dict(map(lambda kv: (kv[0], kv[1].numpy().shape), unitaries.items())))

# initial state
init_state = createState(model, occupied_levels)
state = init_state.numpy().flatten()
print("initial state=", state, ", occupation=", exp.populations(state, model.lindbladian).numpy())

2021-08-17 17:12:46.000056: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-17 17:12:46.000523: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-17 17:12:46.000562: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-17 17:12:46.000604: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-08-17 17:12:46.001489: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-17 17:12:47.312800: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-17 17:1

unitaries:  {'lower-X[0]': (5, 5)}
initial state= [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j] , occupation= [1. 0. 0. 0. 0.]


In [4]:
# time evolution and signal before optimisation
sequence = ["lower-X[0]"] * 3
populations = runTimeEvolutionDefault(exp, init_state, sequence)
plotOccupations(exp, populations, sequence, filename=directory + "/populations_before.png")
signal = generator.generate_signals(gate)[getDrive(model, q1).name]
plotSignal(signal['ts'], signal['values'], directory + "/signal_before.png", spectrum_cut=1e-4)

saving plot in ./output/populations_before.png
saving plot in ./output/signal_before.png


In [5]:
# optimise
optimisable_gates = list(filter(lambda g: g.get_key() != "id[]", gates))

# add all optimisable parameters to a map
drives = filterValues(model.couplings, chip.Drive)
gateset_opt_map = []
for gate in gates:
    for target in gate.targets:
        # TODO: target as index might not always work
        drive = drives[target]
        gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "amp")])
        gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "freq_offset")])
        gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "xy_angle")])
        gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "delta")])
        gateset_opt_map.append([(gate.get_key(), drive.name, "carrier", "framechange")])

callback = lambda fidelity: print(fidelity)
params_before, final_fidelity, params_after = optimise(
    exp, optimisable_gates,
    optimisable_parameters=gateset_opt_map,
    fidelity_fctn=fidelities.state_transfer_infid_set,
    fidelity_params={
        'psi_0': init_state[:active_levels],
        'active_levels': 4,
    },
    callback=callback,
    log_dir=(directory + ("/log_{0:.2f}/".format(t_final * 1e9)))
)
print('before:\n', params_before)
print('after:\n', params_after)
print('fidelity:\n', final_fidelity)

C3:STATUS:Saving as: /home/user/c3/output/log_10.00/c1_state_transfer_infid_set_lbfgs/2021_08_17_T_17_12_59/open_loop.log
0.004581412615738523


/home/user/c3/c3/optimizers/optimizer.py:290: Warning: lower-X[0]-d1-carrier-framechange has no gradient. This might indicate no usage for current experiment.
  warnings.warn(


0.0170485295582542
0.008470928784888199
0.0004632840582079467
0.00045967169506277994
0.00045673573052795735
0.00045673522780076414
0.0004567334568768011
0.00045672029433196926
0.0004566932602612095
before:
 lower-X[0]-d1-gauss-amp               : 500.000 mV 
lower-X[0]-d1-gauss-freq_offset       : -53.000 MHz 2pi 
lower-X[0]-d1-gauss-xy_angle          : -444.089 arad 
lower-X[0]-d1-gauss-delta             : -1.000  
lower-X[0]-d1-carrier-framechange     : 0.000 rad 

after:
 0.00045672029433196926
fidelity:
 lower-X[0]-d1-gauss-amp               : 530.347 mV 
lower-X[0]-d1-gauss-freq_offset       : -53.004 MHz 2pi 
lower-X[0]-d1-gauss-xy_angle          : 12.190 mrad 
lower-X[0]-d1-gauss-delta             : -853.663 m 
lower-X[0]-d1-carrier-framechange     : 0.000 rad 



In [6]:
# time evolution and signal after optimisation
populations = runTimeEvolutionDefault(exp, init_state, sequence)
plotOccupations(exp, populations, sequence, filename=directory + "/populations_after.png")
signal = generator.generate_signals(gate)[getDrive(model, q1).name]
plotSignal(signal['ts'], signal['values'], directory + "/signal_after.png", spectrum_cut=1e-4)

saving plot in ./output/populations_after.png
saving plot in ./output/signal_after.png
